In [1]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)

def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper
    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses
    scraping.contract_addresses.main()

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    

In [2]:
import pandas as pd

with open('gainers.csv') as f:
    df = pd.read_csv(f)



In [5]:
from google.cloud import bigquery
from dotenv import load_dotenv
import os 

load_dotenv()  # Load environment variables from .env file

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]


# client = bigquery.Client()

# query = """
#     SELECT name, SUM(number) as total_people
#     FROM `bigquery-public-data.usa_names.usa_1910_2013`
#     WHERE state = 'TX'
#     GROUP BY name, state
#     ORDER BY total_people DESC
#     LIMIT 20
# """

# query_job = client.query(query)  # Make an API request.

# print("The query data:")
# for row in query_job:
#     # Row values can be accessed by field name or index.
#     print("name={}, count={}".format(row[0], row["total_people"]))

'test seecret'